In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
%matplotlib inline
from datetime import datetime

import geopandas as gp
from shapely import wkt

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
cols_order =  ['id', 'area', '.geo',
               '2021-04-15','2021-04-16','2021-04-18','2021-04-19','2021-04-20','2021-04-22','2021-04-23','2021-04-25','2021-04-26','2021-04-27','2021-04-28','2021-04-29','2021-04-30','2021-05-01',
               '2021-05-02','2021-05-03','2021-05-04','2021-05-07','2021-05-08','2021-05-09','2021-05-10','2021-05-15','2021-05-16','2021-05-17','2021-05-19','2021-05-20','2021-05-21','2021-05-24',
               '2021-05-26','2021-05-27','2021-05-29','2021-06-02','2021-06-03','2021-06-04','2021-06-05','2021-06-06','2021-06-07','2021-06-09','2021-06-10','2021-06-12','2021-06-13','2021-06-16',
               '2021-06-18','2021-06-19','2021-06-20','2021-06-22','2021-06-25','2021-06-27','2021-06-28','2021-07-04','2021-07-05','2021-07-07','2021-07-08','2021-07-09','2021-07-13','2021-07-15',
               '2021-07-17','2021-07-20','2021-07-26','2021-07-27','2021-07-29','2021-07-31','2021-08-01','2021-08-07','2021-08-10','2021-08-11','2021-08-12','2021-08-13','2021-08-23','2021-08-27']
              
df = pd.read_csv("../../common_data/Innopolis/train_dataset_train.csv")[cols_order + ['crop']]
sub = pd.read_csv("../../common_data/Innopolis/test_dataset_test.csv")[cols_order]
renamer = lambda x: x.split('mean_')[-1] if 'nd_mean' in x else x
df.rename(mapper = renamer, axis=1, inplace=True)
sub.rename(mapper = renamer, axis=1, inplace=True)       
df = df[cols_order + ['crop']]
sub = sub[cols_order]

In [ ]:
f = pd.read_csv('../../Ivan/Inno/geo_coords.csv')[['id', 'centroid']]

f['x'] = gp.GeoSeries.from_wkt(f['centroid']).x
f['y'] = gp.GeoSeries.from_wkt(f['centroid']).y

f = f[['id', 'x', 'y']]
f.head()


df = df.merge(f, on = 'id', how = 'left')
sub = sub.merge(f, on = 'id', how = 'left')

In [ ]:
def make_geo_cell(df):

    df['y_05'] = (df['y'] // 1.5).astype(int)
    df['x_05'] = (df['x'] // 1.5).astype(int)

    df['geo_cell'] = df['y_05'].astype(str) + '_' + df['x_05'].astype(str)
    
    df.drop(['y_05', 'x_05'], axis=1, inplace=True)
    
    
make_geo_cell(df)
make_geo_cell(sub)

df['val'] = 1
cols = ['crop','geo_cell', 'id', 'val']
df_pivot = df[cols].pivot_table(index = 'geo_cell', aggfunc = 'mean', values = 'val', columns = 'crop').fillna(0)
df_pivot.columns = [f'crop_{i}' for i in df_pivot.columns]
df_pivot.reset_index(inplace=True)
df.drop('val', axis=1, inplace=True)

In [ ]:
df = df.merge(df_pivot, on = 'geo_cell', how = 'left')
sub = sub.merge(df_pivot, on = 'geo_cell', how = 'left')


In [ ]:
X_array = df['x'].values
Y_array = df['y'].values

def near_feild_fs(df):  
    X_array = df['x'].values
    Y_array = df['y'].values
    f = lambda x : np.sum(abs(X_array - x) <= 0.05)
    df['x_n_near_05'] = df['x'].apply(f)
    f = lambda x : np.sum(abs(X_array - x) <= 0.01)
    df['x_n_near_01'] = df['x'].apply(f)
    df['x_n_near_01_on_05'] = df['x_n_near_01']/df['x_n_near_05'] 
    f = lambda x : np.sum(abs(Y_array - x) <= 0.01)
    df['y_n_near_01'] = df['y'].apply(f)
    f = lambda x : np.sum(abs(Y_array - x) <= 0.05)
    df['y_n_near_05'] = df['y'].apply(f)
    df['y_n_near_01_on_05'] = df['y_n_near_01']/df['y_n_near_05']    
    f = lambda x : np.mean(abs(X_array - x) <= 0.05)
    df['x_n_near_05_mean'] = df['x'].apply(f)
    f = lambda x : np.mean(abs(X_array - x) <= 0.01)
    df['x_n_near_01_mean'] = df['x'].apply(f)
    f = lambda x : np.mean(abs(Y_array - x) <= 0.01)
    df['y_n_near_01_mean'] = df['y'].apply(f)
    f = lambda x : np.mean(abs(Y_array - x) <= 0.05)
    df['y_n_near_05_mean'] = df['y'].apply(f)
    
    
near_feild_fs(df)
near_feild_fs(sub)

In [ ]:
fs2drop = ['geo_cell']

drop_features = ['id', '.geo'] + fs2drop
targets = ['crop']
cat_features = [] 

filtered_features = [i for i in df.columns if (i not in targets and i not in drop_features)]
num_features = [i for i in filtered_features if i not in cat_features]


print('cat_features', cat_features)
print('num_features', len(num_features))
print('targets', targets)

for c in cat_features:
    df[c] = df[c].astype(str)

cat_features []
num_features 86
targets ['crop']


In [ ]:
date_cols = [col for col in df.columns if '2021-' in col]

agg_features = ['month_argmax','month_mean','month_5','month_7','max_mean_diff','month_max_mean_diff','argmin',
                'mean','month_8','var','std','month_var','month_std','month_max','max','month_4','argmax','month_argmin','month_6']

dates_4 = [col for col in date_cols if "-04-" in col]
dates_5 = [col for col in date_cols if "-05-" in col]
dates_6 = [col for col in date_cols if "-06-" in col]
dates_7 = [col for col in date_cols if "-07-" in col]
dates_8 = [col for col in date_cols if "-08-" in col]


def make_stat_agg(df):
    date_cols = [col for col in df.columns if '2021-' in col]
    agg_features = ['month_argmax','month_mean','month_5','month_7','max_mean_diff','month_max_mean_diff','argmin',
                    'mean','month_8','var','std','month_var','month_std','month_max','max','month_4','argmax','month_argmin','month_6']
    dates_4 = [col for col in date_cols if "-04-" in col]
    dates_5 = [col for col in date_cols if "-05-" in col]
    dates_6 = [col for col in date_cols if "-06-" in col]
    dates_7 = [col for col in date_cols if "-07-" in col]
    dates_8 = [col for col in date_cols if "-08-" in col]
    df["mean"] = df[date_cols].values.mean(axis=1)
    df["min"] = df[date_cols].values.min(axis=1)
    df["max"] = df[date_cols].values.max(axis=1)
    df["var"] = df[date_cols].values.var(axis=1)
    df["std"] = df[date_cols].values.std(axis=1)
    df["argmax"] = np.argmax(df[date_cols].values, axis=1)
    df["argmin"] = np.argmin(df[date_cols].values, axis=1)
    df["max_mean_diff"] = df["max"] - df["mean"]
    month_sum = np.array([df[dates_4].values.sum(axis=1), df[dates_5].values.sum(axis=1),
                          df[dates_6].values.sum(axis=1), df[dates_7].values.sum(axis=1),
                          df[dates_8].values.sum(axis=1)]).T
    df["month_mean"] = month_sum.mean(axis=1)
    df["month_min"] = month_sum.min(axis=1)
    df["month_max"] = month_sum.max(axis=1)
    df["month_var"] = month_sum.var(axis=1)
    df["month_std"] = month_sum.std(axis=1)
    df["month_argmax"] = np.argmax(month_sum, axis=1)
    df["month_argmin"] = np.argmin(month_sum, axis=1)
    df["month_max_mean_diff"] = df["month_max"] - df["month_mean"]
    month_sum = pd.DataFrame(month_sum, columns=["month_4", "month_5", "month_6", "month_7", "month_8"])
    df = pd.concat([df, month_sum], axis=1)
    return df, agg_features

filtered_features += agg_features
filtered_features = list(np.unique(filtered_features))

In [ ]:
df = pd.read_csv("../../common_data/Innopolis/train_dataset_livington_v4.csv")
sub = pd.read_csv("../../common_data/Innopolis/test_dataset_livington_v4.csv")

near_feild_fs(df)
near_feild_fs(sub)

In [ ]:
adress_features = ["municipality", "municipality_type", "county", "county_type", "state", "state_type",
                   "ISO3166-2-lvl4", "region"]


def get_county_type(county):
    if "район" in county:
        return "район"
    if "округ" in county:
        return "округ"
    return "другое"
    
def get_state_type(state):
    if "область" in state:
        return "область"
    if "край" in state:
        return "край"
    return "другое"

def get_municipality_type(municipality):
    if "поселение" in municipality:
        return "поселение"
    if "сельсовет" in municipality:
        return "сельсовет"
    return "другое"

def get_adress(df):
    df['municipality'] = df["location"].apply(lambda x: x['address'].get('municipality', 'nan'))
    df['municipality_type'] = df["municipality"].apply(lambda county: get_municipality_type(county))
    df['county'] = df["location"].apply(lambda x: x['address'].get('county', 'nan'))
    df['county_type'] = df["county"].apply(lambda county: get_county_type(county))
    df['state'] = df["location"].apply(lambda x: x['address'].get('state', 'nan'))
    df['state_type'] = df["state"].apply(lambda state: get_state_type(state))
    df['ISO3166-2-lvl4'] = df["location"].apply(lambda x: x['address'].get('ISO3166-2-lvl4', 'nan'))
    df['region'] = df["location"].apply(lambda x: x['address'].get('region', 'nan'))
    df['country_code'] = df["location"].apply(lambda x: x['address'].get('country_code', 'nan'))
    
df[adress_features].sample(3)

In [ ]:
adress_agg_features = ["state_mean", "region_mean", "mean_state_mean_diff", "mean_region_mean_diff",
                       "municipality_mean", "mean_municipality_mean_diff"]

for adress_feature in ["municipality", "state", "region"]:
    
    val_dict = sub.groupby(adress_feature)["mean"].mean().to_dict()
    df[f"{adress_feature}_mean"] = df[adress_feature].apply(lambda x: val_dict.get(x, 0.))
    df[f"mean_{adress_feature}_mean_diff"] = df[f"{adress_feature}_mean"] - df["mean"]
    sub[f"{adress_feature}_mean"] = sub[adress_feature].apply(lambda x: val_dict.get(x, 0.))
    sub[f"mean_{adress_feature}_mean_diff"] = sub[f"{adress_feature}_mean"] - sub["mean"]

df[adress_agg_features].head(2)

In [ ]:
df = pd.read_csv("../../common_data/Innopolis/train_dataset_livington_v4.csv")
sub = pd.read_csv("../../common_data/Innopolis/test_dataset_livington_v4.csv")

In [ ]:
weather_cols = ["tavg", "tmin", "tmax", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun"]

def get_weather_info(df):
    for date_col in tqdm.tqdm(date_cols):
        weather_dict = {wc: [] for wc in weather_cols}
        for i, row in df.iterrows():
            date = date_col.split('-')
            y,m,d = int(date[0]), int(date[1]), int(date[2])
            start = datetime(y, m, d)
            end = datetime(y, m, d)
            
            location = Point(row.x, row.y)
            
            stations = Stations()
            stations = stations.nearby(row.x, row.y)
            station = stations.fetch(10)

            data = Daily(station, start, end)
            data = data.fetch()
            data.fillna(0., inplace=True)
            
            for wc in weather_cols:
                try:
                    weather_dict[wc].append(np.median(data[wc].values))
                except Exception as e:
                    print(e, data)
                    weather_dict[wc].append(0.)
                
        for wc in weather_cols:
            df[f"{date_col}_{wc}"] = weather_dict[wc]

            
weather_date_cols = []
for date_col in date_cols:
    for wc in weather_cols:
        weather_date_cols.append(f"{date_col}_{wc}")


df[weather_date_cols].sample(2)

In [ ]:
cat_features += adress_features
cat_features = list(np.unique(cat_features))

filtered_features += adress_features + adress_agg_features
filtered_features = list(np.unique(filtered_features))

filtered_features += weather_date_cols
filtered_features = list(np.unique(filtered_features))

cat_features

In [ ]:
random_state = 42
n_splits = 3
clfs = []

targets = ['crop']
X = df[filtered_features].drop(targets, axis=1, errors='ignore')
y = df[targets]

kFold_random_state = [42]

N = len(kFold_random_state)*n_splits

for num, random_state in enumerate(kFold_random_state):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for train_index, test_index in kf.split(X):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
        clf = XGBClassifier(objective ='multi:softprob',
                          colsample_bytree = 0.3,
                          subsample = 0.8,
                            
                          eta = 0.001,
                          learning_rate = 0.015,
                          max_depth = 4,
                          alpha = 10,
                          n_jobs = -1, 
                          min_child_weight = 6,
                          reg_alpha=0.1,
                          reg_lambda = 0.1/2,
                          nthread=4,
                          gamma = 0.031,
                          n_estimators = 5_000,
                          early_stopping_rounds=300,
                          eval_metric = 'mlogloss',
                           )

        clf.fit(X = X_train,
                y = y_train,
                verbose = 400,
                eval_set=[(X_test, y_test)],
               )

        
        clfs.append(clf)
        

y_pred = np.zeros((sub.shape[0], 7))
scores = {'multi_logloss':[], 'Precision' : [], 'Recall' : []}
for n, clf in enumerate(clfs):
    y_pred += clf.predict_proba(sub[filtered_features])
    scores['multi_logloss'].append(clf.best_score)
y_pred /= N
print('mean multi_logloss', np.mean(scores['multi_logloss'], dtype = 'float32'), np.std(scores['multi_logloss'], dtype = 'float16'), f'По {len(clfs)} моделям')

In [ ]:
sample_solution = pd.read_csv('../../common_data/Innopolis/sample_solution.csv')
sample_solution['crop'] = np.argmax(y_pred, axis=1)
sample_solution.to_csv('./subs/xgb_more_geo_fs_solution.csv', index=False)